In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import netCDF4 as nc



In [ ]:
# Prediction input from grpahtcast demo 
demo_input_path = ""
demo_input_nc = nc.Dataset(demo_input_path, 'r')
lons_demo = demo_input_nc.variables['lon'][:]
lats_demo = demo_input_nc.variables['lat'][:]
time_demo = demo_input_nc.variables['time'][:]
lsm_demo = demo_input_nc.variables['land_sea_mask'][:]
t2m_demo = demo_input_nc.variables['2m_temperature'][:]
msl_demo = demo_input_nc.variables['mean_sea_level_pressure'][:]
tp_demo = demo_input_nc.variables['total_precipitation_6hr'][:]
tisr_demo = demo_input_nc.variables['toa_incident_solar_radiation'][:]
z_demo = demo_input_nc.variables['geopotential'][:]
u10_demo = demo_input_nc.variables['10m_u_component_of_wind'][:]
v10_demo = demo_input_nc.variables['10m_v_component_of_wind'][:]

1. Geographical Map: Plotting Temperature (2m Temperature)

In [ ]:

# Assuming lons_demo, lats_demo, and t2m_demo are loaded as described

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.coastlines()

# Assuming t2m_demo is 2D (latitude x longitude), and you have a specific time index (e.g., 0)
temperature = t2m_demo[0].squeeze()  # Adjust index for time step as needed and remove singleton dimension
temperature_2d = temperature.reshape(lats_demo.shape)  # Reshape to 2D
contour = ax.contourf(lons_demo, lats_demo, temperature_2d, transform=ccrs.PlateCarree(), cmap='coolwarm')

plt.colorbar(contour, ax=ax, orientation='vertical')
plt.title('2m Temperature')
plt.show()

In [ ]:
# Calculate global mean (simplistically, not accounting for area differences)
global_mean_temp = np.mean(t2m_demo, axis=(1, 2))  # Assuming t2m_demo is (time, lat, lon)

plt.figure(figsize=(10, 5))
plt.plot(time_demo, global_mean_temp)
plt.xlabel('Time')
plt.ylabel('Global Mean Temperature')
plt.title('Global Mean 2m Temperature Over Time')
plt.show()

In [ ]:
# Flatten the temperature data for a specific time step to get its distribution
temp_flat = tp_demo[0].flatten()  # Adjust index for time step as needed

plt.figure(figsize=(10, 5))
plt.hist(temp_flat, bins=10, color='blue', edgecolor='black')
plt.title('Distribution of 2m Temperature')
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.show()


In [ ]:

# Extract variables and compute mean values for the first time step
t2m_mean = np.mean(demo_input_nc.variables['2m_temperature'][0, :, :])
msl_mean = np.mean(demo_input_nc.variables['mean_sea_level_pressure'][0, :, :])
tp_mean = np.mean(demo_input_nc.variables['total_precipitation_6hr'][0, :, :])
u10_mean = np.mean(demo_input_nc.variables['10m_u_component_of_wind'][0, :, :])

# Close the dataset
demo_input_nc.close()

# Prepare data for radar chart
labels = np.array(['2m Temp', 'MSL Pressure', 'Precipitation', 'Wind Speed'])
stats = np.array([t2m_mean, msl_mean, tp_mean, u10_mean])

# Radar chart setup
angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
stats = np.concatenate((stats,[stats[0]]))  # Create a closed loop
angles += angles[:1]  # Complete the circle for the radar chart

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
ax.fill(angles, stats, color='red', alpha=0.25)
ax.plot(angles, stats, color='red', linewidth=2)  # Draw the outline of our data
ax.set_yticklabels([])
ax.set_xticks(angles[:-1])
ax.set_xticklabels(labels)

plt.title('Climate Variables Overview')
plt.show()
